## 1. Hidden Markov Model (HMM)

Let $Z_t$ be the hidden state at time $t$ where $Z_t \in \{0,1,\dots,K-1\}$, and let $X_t$ be the observed feature vector.

1. **Transition (Markov property)**  
$$
P(Z_t=j\mid Z_{t-1}=i)=A_{ij},\qquad
A=\begin{bmatrix}
a_{0,0}&\cdots&a_{0,K-1}\\
\vdots&\ddots&\vdots\\
a_{K-1,0}&\cdots&a_{K-1,K-1}
\end{bmatrix}.
$$

2. **Emission (Gaussian observation model)**  
$$
P(X_t\mid Z_t=k)=\mathcal N(X_t\mid \mu_k,\Sigma_k).
$$

3. **Initial distribution (stationary)**  
$$
\pi=
\begin{bmatrix}
\pi_0\\
\vdots\\
\pi_{K-1}
\end{bmatrix},
\qquad
\pi=A^\top \pi,
\qquad
\sum_{k=0}^{K-1}\pi_k=1.
$$

4. **Joint probability**
$$
\begin{aligned}
P(X_{1:T},Z_{1:T})
&=
\underbrace{P(Z_1)}_{\pi_{Z_1}}
\cdot
\underbrace{P(X_1 \mid Z_1)}_{\text{Emission at } t=1}
\cdot
\prod_{t=2}^{T}
\underbrace{P(Z_t \mid Z_{t-1})}_{A_{Z_{t-1}, Z_t}}
\cdot
\underbrace{P(X_t \mid Z_t)}_{\text{Emission at } t}
\\
&=
\pi_{Z_1}\,P(X_1\mid Z_1)\,
\prod_{t=2}^{T} A_{Z_{t-1},Z_t}\,P(X_t\mid Z_t).
\end{aligned}
$$


---

### Learning & Inference

#### Goal 1: Learn parameters $\theta=\{\pi,A,\mu_k,\Sigma_k\}$

We maximize the data likelihood:
$$
\hat\theta=\arg\max_{\theta} P(X_{1:T}\mid \theta)
=
\arg\max_{\theta}\sum_{Z_{1:T}} P(X_{1:T},Z_{1:T}\mid \theta).
$$
Solved by **Baum–Welch (EM)**.

**EM idea**  
E-step: compute how likely each state/transition was.  
M-step: update parameters using those likelihood weights.

**Model**
$$
P(X_{1:T},Z_{1:T}\mid\theta)
=
\pi_{Z_1}\prod_{t=2}^{T}A_{Z_{t-1},Z_t}\prod_{t=1}^{T}b_{Z_t}(X_t),
\qquad
b_k(x)=\mathcal N(x\mid\mu_k,\Sigma_k).
$$

**E-step (forward–backward)**
$$
\alpha_1(k)=\pi_k b_k(X_1),\qquad
\alpha_{t+1}(j)=\Big(\sum_{i=0}^{K-1}\alpha_t(i)A_{ij}\Big)\,b_j(X_{t+1}).
$$
$$
\beta_T(k)=1,\qquad
\beta_t(i)=\sum_{j=0}^{K-1}A_{ij}\,b_j(X_{t+1})\,\beta_{t+1}(j).
$$
$$
\gamma_t(k)=P(Z_t=k\mid X_{1:T})
=
\frac{\alpha_t(k)\beta_t(k)}{\sum_{\ell=0}^{K-1}\alpha_t(\ell)\beta_t(\ell)}.
$$
$$
\xi_t(i,j)=P(Z_t=i,Z_{t+1}=j\mid X_{1:T})
=
\frac{\alpha_t(i)A_{ij}b_j(X_{t+1})\beta_{t+1}(j)}
{\sum_{p=0}^{K-1}\sum_{q=0}^{K-1}\alpha_t(p)A_{pq}b_q(X_{t+1})\beta_{t+1}(q)}.
$$

**M-step (update rules)**
$$
\pi_k^{new}=\gamma_1(k),
\qquad
A_{ij}^{new}=\frac{\sum_{t=1}^{T-1}\xi_t(i,j)}{\sum_{t=1}^{T-1}\gamma_t(i)}.
$$
$$
\mu_k^{new}=\frac{\sum_{t=1}^{T}\gamma_t(k)X_t}{\sum_{t=1}^{T}\gamma_t(k)}.
$$
$$
\Sigma_k^{new}
=
\frac{\sum_{t=1}^{T}\gamma_t(k)(X_t-\mu_k^{new})(X_t-\mu_k^{new})^\top}
{\sum_{t=1}^{T}\gamma_t(k)}.
$$

---

#### Goal 2: Posterior state probabilities (smoothing)

For each time $\tau$:
$$
\gamma_\tau(k)=P(Z_\tau=k\mid X_{1:T},\hat\theta)
=
\frac{\alpha_\tau(k)\beta_\tau(k)}{\sum_{j=0}^{K-1}\alpha_\tau(j)\beta_\tau(j)}.
$$

**One-step-ahead state probability (used in code)**
$$
P(Z_{T+1}=\cdot\mid X_{1:T},\hat\theta)=\gamma_T^\top A.
$$

---

#### Goal 3: Next-step mean/std and downside risk (matches your code)

We fit the HMM on standardized data $X'$:
$$
X'=\frac{X-m}{s}
\quad\Longleftrightarrow\quad
X=m+sX',
\qquad
m=\texttt{scaler.mean\_},\; s=\texttt{scaler.scale\_}.
$$

If state $k$ has (in standardized space)
$$
X'\mid (Z=k)\sim \mathcal N(\mu_k',(\sigma_k')^2),
$$
then in original units:
$$
\mu_k=m+s\mu_k',
\qquad
\sigma_k=s\sigma_k',
\qquad
\sigma_k'=\sqrt{(\sigma_k')^2}.
$$

**Downside second moment for a Gaussian (threshold $0$)**  
Let $X\sim\mathcal N(\mu,\sigma^2)$ and define
$$
z=\frac{0-\mu}{\sigma}=-\frac{\mu}{\sigma}.
$$
Then
$$
\mathbb E\!\left[X^2\mathbf 1_{\{X<0\}}\right]
=
(\mu^2+\sigma^2)\Phi(z)-\mu\sigma\,\phi(z),
$$
where $\Phi$ is the standard normal CDF and $\phi$ is the PDF.

For each state $k$, define
$$
d_k=(\mu_k^2+\sigma_k^2)\Phi\!\Big(-\frac{\mu_k}{\sigma_k}\Big)
-\mu_k\sigma_k\,\phi\!\Big(-\frac{\mu_k}{\sigma_k}\Big).
$$

**Next-state probabilities**  
If $\mathbf p_t$ is the current state probability row vector, then
$$
\mathbf p_{t+1}=\mathbf p_t A.
$$
(In code this is `next_states_prob`.)

**Expected mean (next step)**
$$
\mathbb E[X_{t+1}\mid X_{1:t}]
=
\sum_{k=0}^{K-1} p_{t+1}(k)\mu_k.
$$

**Expected variance and std (next step)**  
First compute the mixture second moment:
$$
\mathbb E[X_{t+1}^2\mid X_{1:t}]
=
\sum_{k=0}^{K-1} p_{t+1}(k)\big(\sigma_k^2+\mu_k^2\big).
$$
Then
$$
\mathrm{Var}(X_{t+1}\mid X_{1:t})
=
\mathbb E[X_{t+1}^2\mid X_{1:t}]
-
\Big(\mathbb E[X_{t+1}\mid X_{1:t}]\Big)^2,
$$
$$
\mathrm{Std}(X_{t+1}\mid X_{1:t})
=
\sqrt{\max(\mathrm{Var}(\cdot),\varepsilon)}.
$$

**Expected downside deviation (next step, target $0$)**  
Mix the downside second moments:
$$
\mathbb E[X_{t+1}^2\mathbf 1_{\{X_{t+1}<0\}}\mid X_{1:t}]
=
\sum_{k=0}^{K-1} p_{t+1}(k)d_k.
$$
Define downside std:
$$
\mathrm{DownStd}(X_{t+1}\mid X_{1:t})
=
\sqrt{\max\!\Big(\sum_{k=0}^{K-1} p_{t+1}(k)d_k,\varepsilon\Big)}.
$$


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn.hmm import GaussianHMM, GMMHMM
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import yfinance as yf

import warnings
warnings.filterwarnings('ignore')


def data_preparation(tickers, period="30y", window_size=5, ma_size=20, interval="1d"):
    assets = {}
    assets_oo = {}
    for ticker in tickers:
        assets[ticker]    = yf.download(ticker, period=period, interval=interval, auto_adjust=True, progress=False)["Close"]
        assets_oo[ticker] = yf.download(ticker, period=period, interval=interval, auto_adjust=True, progress=False)["Close"].pct_change().shift(-1)  # For calculating entry position at t
        assets[ticker].columns    = [f"{ticker}_Close"]
        assets_oo[ticker].columns = [f"{ticker}_oo"]
        assets[ticker][f"{ticker}_ret"]           = assets[ticker][f"{ticker}_Close"].pct_change()
        assets[ticker][f"{ticker}_std"]           = assets[ticker][f"{ticker}_ret"].rolling(window=window_size).std()
        assets[ticker][f"{ticker}_ewma_ret"]      = assets[ticker][f"{ticker}_Close"].ewm(span=window_size, adjust=False).mean().pct_change()
        assets[ticker][f"{ticker}_ewma_down_std"] = assets[ticker][f"{ticker}_ewma_ret"].where(assets[ticker][f"{ticker}_ewma_ret"] < 0, 0).rolling(window=ma_size).std()
    rets_oo = pd.concat([assets_oo[ticker] for ticker in tickers], axis=1)
    df = pd.concat([assets[ticker] for ticker in tickers] + [rets_oo], axis=1, join="inner").dropna()
    return df


def num_states_selection(tickers, df, n_states, past_data_size=int(252*3)):
    print(f"Number of states: {n_states}, Number of years data: {int(past_data_size/252)}")
    for ticker in tickers:
        X_raw = df[[f"{ticker}_ewma_ret"]].values[:past_data_size+1]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_raw)
        model = GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=500, tol=1e-3, min_covar=1e-3).fit(X_train)
        #model = GMMHMM(n_components=n_states, n_mix=2, covariance_type="diag", n_iter=500, tol=1e-3, min_covar=1e-3).fit(X_train)
        aic_score = round(model.aic(X_train), 4)
        bic_score = round(model.bic(X_train), 4)
        print(f"{ticker}: AIC: {aic_score}, BIC:{bic_score}")
    print("\n")


# 1. Try n_states=3/4
def hmm_weight(tickers, df, n_states=4, past_data_size=int(252*3), fit_dist_size=int(252/3), accumulation_size=40, gap_prob=0.5, ret_thres=-0.005, capped_score=1, seed=42, n_iters=500, n_restarts=40):
    X_raws, X_raw_ret, X_raw_std, scalers, models, scores, weight = {}, {}, {}, {}, {}, {}, {}
    accumulation_number = 0
    weight_list = np.zeros(shape=(len(df), len(tickers)))
    for ticker in tickers:
        X_raws[ticker] = df[[f"{ticker}_ewma_ret"]].values
        X_raw_ret[ticker] = df[[f"{ticker}_ret"]].values
        X_raw_std[ticker] = df[[f"{ticker}_ewma_down_std"]].values

    for t in range(past_data_size, len(df)):
        sum_scores = 0
        for ticker in tickers:
            X_raw_trains = X_raws[ticker][t-past_data_size+1:t+1]

            # Training Day 
            if accumulation_number == 0: 
                scalers[ticker] = StandardScaler()
                X_trains = scalers[ticker].fit_transform(X_raw_trains)
                hmm_models = [GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=n_iters, tol=1e-3, min_covar=1e-3, random_state=seed+i).fit(X_trains) for i in range(n_restarts)]
                #hmm_models = [GMMHMM(n_components=n_states, n_mix=2, covariance_type="diag", n_iter=100, tol=1e-3, min_covar=1e-3, random_state=seed+i).fit(X_trains) for i in range(n_restarts)]
                models[ticker] = max(hmm_models, key=lambda m: m.score(X_trains))

            # Non-Training Day
            else:
                X_trains = scalers[ticker].transform(X_raw_trains)

            state_posterior_prob = models[ticker].predict_proba(X_trains)
            next_states_prob = state_posterior_prob[-1] @ models[ticker].transmat_

            """            
            states = models[ticker].predict(X_trains)
            mean               = [X_raw_trains[-fit_dist_size: ][states[-fit_dist_size: ] == s, 0].mean()                     for s in range(n_states)] 
            var                = [X_raw_trains[-fit_dist_size: ][states[-fit_dist_size: ] == s, 0].var()                      for s in range(n_states)]
            down_second_moment = [np.mean(np.minimum(0.0, X_raw_trains[-fit_dist_size:][states[-fit_dist_size:] == s, 0])**2) for s in range(n_states)]
            
            gamma_fit = state_posterior_prob[-fit_dist_size:]
            #X_raw_fit = X_raw_trains[-fit_dist_size:, 0]
            X_raw_fit = X_raw_ret[ticker][t-fit_dist_size+1:t+1, 0]
            mean               = (gamma_fit.T @ X_raw_fit)                          / (gamma_fit.sum(axis=0) + 1e-12)
            var                = (gamma_fit.T @ ((X_raw_fit[:, None] - mean) ** 2)) / (gamma_fit.sum(axis=0) + 1e-12)
            down_second_moment = (gamma_fit.T @ (np.minimum(0.0, X_raw_fit) ** 2))  / (gamma_fit.sum(axis=0) + 1e-12)

            expected_mean_next_state     = np.array(mean) @ next_states_prob
            expected_std_next_state      = np.sqrt((var + (mean - expected_mean_next_state) ** 2) @ next_states_prob + 1e-12)
            expected_down_std_next_state = np.sqrt(np.array(down_second_moment) @ next_states_prob + 1e-12)
            """
            
            #mean               = scalers[ticker].mean_ + scalers[ticker].scale_ * models[ticker].means_
            #std                = scalers[ticker].scale_ * np.sqrt(models[ticker].covars_)
            mean               = scalers[ticker].mean_[0] + scalers[ticker].scale_[0] * models[ticker].means_.ravel()
            std                = scalers[ticker].scale_[0] * np.sqrt(models[ticker].covars_.ravel())
            z                  = -mean / np.maximum(std, 1e-12)
            down_second_moment = (mean**2 + std**2) * norm.cdf(z) - (mean * std) * norm.pdf(z)

            expected_mean_next_state     = float(next_states_prob @ mean)
            expected_var_next_state      = float(next_states_prob @ (std**2 + mean**2) - expected_mean_next_state**2)
            expected_std_next_state      = float(np.sqrt(max(expected_var_next_state, 1e-12)))
            expected_down_std_next_state = float(np.sqrt(max(next_states_prob @ down_second_moment, 1e-12)))

            expected_sharpe_next_state   = expected_mean_next_state / expected_std_next_state
            expected_sortino_next_state  = expected_mean_next_state / expected_down_std_next_state

            crash_state_idx = np.argmin(mean)
            crash_state_mean = mean[crash_state_idx]
            highest_prob_next_state_idx = np.argmax(next_states_prob)
            scores[ticker] = expected_sortino_next_state if (expected_sortino_next_state > 0 and next_states_prob[highest_prob_next_state_idx] > next_states_prob[crash_state_idx] + gap_prob) else -np.inf

            """
            if crash_state_mean > 0:
                scores[ticker] = expected_sortino_next_state
            else:
                if next_states_prob[highest_prob_next_state_idx] > next_states_prob[crash_state_idx] + gap_prob:
                    if expected_mean_next_state > 0:
                        scores[ticker] = expected_sortino_next_state
                    elif (expected_mean_next_state > ret_thres and expected_down_std_next_state < float(X_raw_std[ticker][t].item())):
                        scores[ticker] = min((expected_mean_next_state - ret_thres) / expected_down_std_next_state, capped_score)
                    else:
                        scores[ticker] = -np.inf
                else:
                    scores[ticker] = -np.inf         
            print(
                ticker,
                "mean: ", round(expected_mean_next_state, 4),
                "down_std: ", round(expected_down_std_next_state, 4),
                "ma_std: ", round(float(X_raw_std[ticker][t].item()), 4),
                "score: ", scores[ticker]
            )"""
            if np.isfinite(scores[ticker]): sum_scores += scores[ticker]

        for ticker_col, ticker in enumerate(tickers):
            if np.isfinite(scores[ticker]): weight_list[t][ticker_col] = scores[ticker] / sum_scores
        
        print("weight at ", t, ": ", weight_list[t])
        accumulation_number += 1
        if accumulation_number == accumulation_size: 
            accumulation_number = 0;
    
    for ticker in tickers:
        weight[ticker] = pd.Series(weight_list[:, tickers.index(ticker)], index=df.index)
    df["port_oo"] = sum([weight[ticker] * df[f"{ticker}_oo"] for ticker in tickers])
    ret_df = df[[f"{ticker}_oo" for ticker in tickers] + ["port_oo"]].iloc[past_data_size:]
    cash_proportion = (weight_list[past_data_size:].sum(axis=1) == 0).mean()
    print(f"Cash Proportion: {cash_proportion:.2%}\n")
    return ret_df


def overall_return_matrics(returns, window=252):
    annual_mean = returns.mean() * window
    annual_std = returns.std() * np.sqrt(window)
    annual_down_std = np.sqrt(np.mean(np.minimum(0.0, returns)**2)) * np.sqrt(window)
    annual_sharpe = annual_mean / annual_std
    annual_sortino = annual_mean / annual_down_std

    rolling_mean = returns.rolling(window).mean()
    rolling_std = returns.rolling(window).std()
    rolling_sharpe = np.sqrt(window) * rolling_mean / rolling_std

    cumprod = (1 + returns).cumprod()
    cumprod_max = cumprod.cummax()
    dd = (cumprod - cumprod_max) / cumprod_max
    max_dd = dd.min()

    return cumprod, rolling_mean, rolling_std, rolling_sharpe, annual_mean, annual_std, annual_sharpe, annual_sortino, max_dd
        

In [ ]:
tickers = ["QQQ", "GLD", "CHFJPY=X"]
df = data_preparation(tickers)
for yr in range(2,6):
    for n in range(2,6):
        num_states_selection(tickers, df, n, past_data_size=int(252*yr))

In [ ]:
ret_df = hmm_weight(tickers, df)
cumprods, rolling_means, rolling_stds, rolling_sharpes, tot_means, tot_stds, tot_sharpes, tot_sortinos, max_dds = {}, {}, {}, {}, {}, {}, {}, {}, {}
port_cumprod, port_rolling_mean, port_rolling_std, port_rolling_sharpe, port_tot_mean, port_tot_std, port_tot_sharpe, port_tot_sortino, port_max_dd = overall_return_matrics(ret_df["port_oo"])
for ticker in tickers:
    cumprods[ticker], rolling_means[ticker], rolling_stds[ticker], rolling_sharpes[ticker], tot_means[ticker], tot_stds[ticker], tot_sharpes[ticker], tot_sortinos[ticker], max_dds[ticker] = overall_return_matrics(ret_df[f"{ticker}_oo"])

# -----------------------------
# 1) Equity curve (cumprod)
# -----------------------------
plt.figure(figsize=(12, 6))
plt.plot(port_cumprod.index, port_cumprod.values, label="Portfolio")
for ticker in tickers:
    plt.plot(cumprods[ticker].index, cumprods[ticker].values, label=ticker)
plt.title("Equity Curve (Cumprod)")
plt.xlabel("Date")
plt.ylabel("Equity (Start = 1.0)")
plt.legend()
plt.grid(True)
plt.show()

# -----------------------------
# 2) Portfolio rolling sharpe
# -----------------------------
plt.figure(figsize=(12, 6))
plt.plot(port_rolling_sharpe.index, port_rolling_sharpe.values, label="port_rolling_sharpe")
plt.title("Portfolio Rolling Sharpe")
plt.xlabel("Date")
plt.legend()
plt.grid(True)
plt.show()

# -----------------------------
# 3) Summary table: mean / std / sharpe / max_dd
# -----------------------------
summary_df = pd.DataFrame({
        "mean":   [port_tot_mean]   + [tot_means[ticker]   for ticker in tickers],
        "std":    [port_tot_std]    + [tot_stds[ticker]    for ticker in tickers],
        "sharpe": [port_tot_sharpe] + [tot_sharpes[ticker] for ticker in tickers],
        "sortino":[port_tot_sortino]+ [tot_sortinos[ticker]for ticker in tickers],
        "max_dd": [port_max_dd]     + [max_dds[ticker]     for ticker in tickers],
    }, index=["Portfolio"] + list(tickers),
)

display(summary_df)

